In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from glob import glob
from tqdm import tqdm
import random
from PIL import Image
import PIL

from scipy.ndimage.filters import gaussian_filter

from sklearn.utils import shuffle
from makizoo.backbones.resnetv1 import ResNet18
from makizoo.backbones.mobilenetv2 import MobileNetV2_1_4, MobileNetV2_1_0, MobileNetV2_0_75
from makiflow.layers import *
from makiflow.models.regressor import Regressor, AbsTrainer, MseTrainer
from makiflow.generators.classification import cycle_generator

from makiflow import set_main_gpu

set_main_gpu(0)

In [ ]:
config_data = {'train': [], 'test': []}
path_exp = 'weights_regres'

SIZE_HW = (336, 336)

path_to_data = 'expand_double_modes' #'/content/drive/MyDrive/Neural network/work/beams/data'
folders_path = sorted(glob(path_to_data + '/*'), key=lambda x: float(x.split('/')[-1]))
Xtrain = []
Xtest = []

Ytrain = []
Ytest = []
pred2param = {}

test_div = 0.8
print(len(folders_path))
for i, single_folder in enumerate(folders_path):
    # Single folder - 1 class
    path_images = shuffle(glob(single_folder + '/*'))
    num_class = float(single_folder.split('/')[-1])
    size = len(path_images)
    iterator = tqdm(enumerate(path_images))
    for j, single_img_path in iterator:
        readed_img = cv2.resize(cv2.imread(single_img_path)[..., ::-1], SIZE_HW)
        if int(size * test_div) > j:
            # train
            Xtrain.append(readed_img)
            Ytrain.append(num_class)
            config_data['train'] += [single_img_path.split('/')[-1]]
        else:
            # test
            Xtest.append(readed_img)
            Ytest.append(num_class)
            config_data['test'] += [single_img_path.split('/')[-1]]
    iterator.close()

with open(f'{path_exp}/config_train_test.json', 'w') as fp:
    json.dump(config_data, fp)
    
print('train : ', len(Ytrain))
print('test: ', len(Ytest))

In [ ]:
coffe_norm = lambda x: (x - 128.0) / 128.0

#Xtrain = coffe_norm(np.array(Xtrain)).astype(np.float32)
#Xtest = coffe_norm(np.array(Xtest)).astype(np.float32)

Ytrain = np.array(Ytrain).astype(np.float32)
Ytest = np.array(Ytest).astype(np.float32)

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(Xtrain[2])

In [ ]:
BATCH_SIZE = 64

In [ ]:
id_num = 0
def block(x, in_f, out_f):
    global id_num
    
    id = id_num
    SX = x
    FX = x
    
    FX = ConvLayer(kw=3, kh=3, in_f=in_f, out_f=out_f, name=f'conv_main_{id}_1', activation=None)(FX)
    FX = BatchNormLayer(D=out_f, name=f'bn_main_{id}_1')(FX)
    FX = ActivationLayer(name=f'act_1_{id}')(FX)
    
    FX = ConvLayer(kw=3, kh=3, in_f=out_f, out_f=out_f, name=f'conv_main_{id}_2', activation=None)(FX)
    FX = BatchNormLayer(D=out_f, name=f'bn_main_{id}_2')(FX)
    FX = ActivationLayer(name=f'act_2_{id}')(FX)
    
    if in_f != out_f:
        SX = BatchNormLayer(D=in_f, name=f'bn_skip_{id}_1')(SX)
        SX = ConvLayer(kw=1, kh=1, in_f=in_f, out_f=out_f, name=f'conv_skip_{id}', activation=None)(SX)
    
    x_sum = SumLayer(name=f'sum_{id}')([FX, SX])
    id_num += 1
    return x_sum

In [ ]:
custom_backbone = False

if custom_backbone:
    inp = InputLayer(input_shape=[None, SIZE_HW[0], SIZE_HW[1], 3], name='input')
    x = block(inp, in_f=3, out_f=64)
    x = MaxPoolLayer(name='mp1')(x)
    # 168
    x = block(x, in_f=64, out_f=128)
    x = block(x, in_f=128, out_f=128)
    x = MaxPoolLayer(name='mp2')(x)
    # 84
    x = block(x, in_f=128, out_f=128)
    x = block(x, in_f=128, out_f=128)
    x = MaxPoolLayer(name='mp3')(x)
    # 42
    x = block(x, in_f=128, out_f=256)
    x = block(x, in_f=256, out_f=256)
    x = MaxPoolLayer(name='mp4')(x)
    # 21
    x = block(x, in_f=256, out_f=512)
    x = block(x, in_f=512, out_f=512)
    #x = MaxPoolLayer(name='mp5')(x)
    # 10
else:
    inp, x = ResNet18([BATCH_SIZE, SIZE_HW[0], SIZE_HW[1], 3])

x = GlobalAvgPoolLayer('flat')(x)
x = DropoutLayer(name='drop_final', p_keep=0.7)(x)
x = DenseLayer(in_d=x.get_shape()[-1], out_d=1, use_bias=False, activation=None, name='regression_out')(x)

In [ ]:
model = Regressor([inp], [x], name='MakiResNet')
sess = tf.Session()
model.set_session(sess)

In [ ]:
trainer = MseTrainer(model, [inp])
trainer.compile()

In [ ]:
len(Ytrain)

In [ ]:
global_step = tf.Variable(0, dtype=tf.int32)
lr = tf.train.exponential_decay(5e-3, global_step, decay_steps=(len(Ytrain) // BATCH_SIZE) * 2, decay_rate=0.92)
opt = tf.train.AdamOptimizer(lr)
sess.run(tf.variables_initializer([global_step]))

In [ ]:
def generator_wrapper(gen_data, *args):
    while True:
        X_data, Y_data = next(gen_data)
        X_data = X_data[0]
        Y_data = Y_data[0]
        """
        rotate_angle = np.random.uniform(-90, 90, size=(len(X_data))).astype(np.float32)
        #X_data = [ndimage.rotate(X_data[i], rotate_angle[i], reshape=False) for i in range(len(X_data))]
        X_data = [
            np.array(
                Image.fromarray(X_data[i]).rotate(rotate_angle[i], resample=Image.BICUBIC).getdata()
            ).reshape(SIZE_HW[0], SIZE_HW[1], 3)
            for i in range(len(X_data))
        ]
        
        ra = random.random()
        type_rotate = None
        
        if ra > 0.3 and ra < 0.5:
            type_rotate = cv2.ROTATE_180
        elif ra > 0.5 and ra < 0.7:
            type_rotate = cv2.ROTATE_90_CLOCKWISE
        elif ra > 0.7:
            type_rotate = cv2.ROTATE_90_COUNTERCLOCKWISE
        
        if type_rotate is not None:
            X_data = [cv2.rotate(X_data[i], type_rotate) for i in range(len(X_data))]
        """
        #if random.random() > 0.6:
        #    X_data = [gaussian_filter(X_data[i], sigma=1.5) for i in range(len(X_data))]
        
        X_data = coffe_norm(np.asarray(X_data)).astype(np.float32, copy=False)
        
        yield (X_data,), (Y_data, )


gen = generator_wrapper(cycle_generator(Xtrain, Ytrain.reshape(-1, 1), BATCH_SIZE))

In [ ]:
xxx = next(gen)[0]

In [ ]:
plt.imshow(xxx[0][8])

In [ ]:
trainer.set_common_l2_weight_decay(1e-5)

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def predict(data_pr):
    output_mf = x
    ans = [
        output_mf.eval(feed_dict={inp: data_pr[i*BATCH_SIZE:(i+1)*BATCH_SIZE]}, sess=sess) 
        for i in range(len(data_pr)//BATCH_SIZE)
    ]
    ans = np.concatenate(ans, axis=0).reshape(-1)
    
    return ans.astype(np.float32, copy=False)

In [ ]:
abs_mean_list = []

for _ in range(20):
    info1 = trainer.fit_generator(
        gen, optimizer=opt, epochs=1, 
        iter=len(Xtrain)//(2*BATCH_SIZE), print_period=150, global_step=global_step
    )
    
    predictions = predict(coffe_norm(np.array(Xtest)))
    predictions_train = predict(coffe_norm(np.array(Xtrain)))
    
    acc_float = np.mean(np.power(predictions - Ytest[:len(predictions)].reshape(-1), 2))
    print('acc test: ', acc_float)
    print('mean pred: ', predictions.mean())
    print('--------------')
    print('mse loss: ', np.mean((predictions_train - Ytrain[:len(predictions_train)].reshape(-1)) ** 2))
    print('mean pred train: ', predictions_train.mean())
    print('train pred: ', predictions_train)
    
    abs_mean_list.append(acc_float)
    print('pred: ', predictions[BATCH_SIZE:].reshape(-1))
    print('labels: ', Ytest[BATCH_SIZE:].reshape(-1))
    
    #predictions = model.predict(coffe_norm(np.array(Xtrain)))
    #predictions = np.argmax(predictions, axis=-1)
    #print('acc train: ', np.mean(predictions == Ytrain.reshape(-1)))

In [ ]:
assert 1 == 3

In [ ]:
#predictions = [
#    model.predict(
#        coffe_norm(np.array(Xtest)[i*BATCH_SIZE:(i+1)*BATCH_SIZE])
#    ) 
#    for i in [len(Xtest)//BATCH_SIZE-1] #range(len(Xtest)//BATCH_SIZE)
#]
predictions = predict(Xtest)
#predictions = np.concatenate(predictions, axis=0).reshape(-1)
acc_float = np.mean(np.power(predictions - Ytest[:len(predictions)].reshape(-1), 2))
print('acc test: ', acc_float)

In [ ]:
predictions[6000:].reshape(-1)

In [ ]:
Ytest[3000:3100].reshape(-1)

In [ ]:
plt.plot(info1[trainer.TRAINING_LOSS])

In [ ]:
predictions = model.predict(coffe_norm(np.array(Xtrain)))
predictions = np.argmax(predictions, axis=-1)
print(np.mean(predictions == Ytrain.reshape(-1)))

In [ ]:
assert 1 == 3

In [ ]:
output_mf = model.get_node('flat')

In [ ]:
ans = [output_mf.eval(feed_dict={inp: Xtest[i*BATCH_SIZE:(i+1)*BATCH_SIZE]}, sess=sess) for i in range(len(Xtest)//BATCH_SIZE)]

In [ ]:
ans = np.concatenate(ans, axis=0)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
pca = PCA()
reduced = pca.fit_transform(ans)
plt.scatter(reduced[:,0],reduced[:,1] ,s=100,c=Ytest[:ans.shape[0]],alpha=0.5)
plt.savefig('pca.jpg')

In [ ]:
tsne = TSNE()
Z = tsne.fit_transform(ans)
plt.scatter(Z[:,0],Z[:,1] ,s=100,c=Ytest[:ans.shape[0]],alpha=0.5)
plt.savefig('tsne.jpg')